# Debug

test 1

In [ ]:
jsoned={'name': 'retrieval', 'parameters': {'search_query': 'How does REACT differ from Chain-of-thought?'}}
def find_function_object(json_data):
    """
    Recursively searches for a dictionary that contains both 'name' and 'arguments' keys within nested JSON.
    
    :param json_data: The JSON data to search through.
    :return: The dictionary that contains 'name' and 'arguments' keys or None if not found.
    """
    if isinstance(json_data, dict):  # If the current data is a dictionary
        # Check if the dictionary has both 'name' and 'arguments' keys
        if 'name' in json_data and 'arguments' in json_data:
            arguments = json_data['arguments']
            # check if parameters is a key value pair dictionary
            if isinstance(arguments, dict):
                for key in arguments:
                    # Do not support nested dictionaries in the parameters
                    if isinstance(arguments[key], dict):
                        return None
            return {'name': json_data['name'], 'arguments': arguments}

        # or 'name' and 'parameters' keys
        if 'name' in json_data and 'parameters' in json_data:
            arguments = json_data['parameters']
            # check if parameters is a key value pair dictionary
            if isinstance(arguments, dict):
                for key in arguments:
                    # Do not support nested dictionaries in the parameters
                    if isinstance(arguments[key], dict):
                        return None
            return {'name': json_data['name'], 'arguments': arguments}

        # Otherwise, iterate over the values and continue the search
        for key in json_data:
            result = find_function_object(json_data[key])
            if result:
                return result

    elif isinstance(json_data, list):  # If the current data is a list
        # Iterate over each item in the list and continue the search
        for item in json_data:
            result = find_function_object(item)
            if result:
                return result
            
    return None  # Return None if no matching dictionary is found

result = find_function_object(jsoned)
print(result)

In [ ]:
def merge_system_messages(messages, system_message):
    last_system_message_index = -1
    i = len(messages) - 1
    # merge system messages from the back to front
    while i >= 0:
        if messages[i]["role"].lower() == "system":
            if last_system_message_index == -1:
                # first system message from the back found
                last_system_message_index = i
                messages[i]["content"] += " " + system_message['content']
            else:
                # next system message from the back found
                messages[i]["content"] += " " + messages[last_system_message_index]['content']
                messages.pop(last_system_message_index)
                last_system_message_index = i
        i -= 1
    return messages

messages = [
    {"role": "system","content":"this is system_message 1."},
    {"role": "system","content":"this is system_message 2."},
    {"role": "user","content":"this is user_message."},
    {"role": "assistant","content":""}
    ]
last_system_message = {"role": "system","content":"this is system_message 3."}
messages=merge_system_messages(messages, last_system_message)
messages

In [ ]:
def merge_system_messages(messages, system_message):
    last_system_message_index = -1
    i = len(messages) - 1
    # merge system messages from the back to front
    while i >= 0:
        if messages[i]["role"].lower() == "system":
            if last_system_message_index == -1:
                # first system message from the back found
                last_system_message_index = i
                messages[i]["content"] += " " + system_message['content']
            else:
                # next system message from the back found
                messages[i]["content"] += " " + messages[last_system_message_index]['content']
                messages.pop(last_system_message_index)
                last_system_message_index = i
        i -= 1
    if last_system_message_index == -1:
        # no system message found
        messages.insert(0,system_message)
    return messages

messages = [
    {"role": "user","content":"this is user_message."},
    {"role": "assistant","content":""}
    ]
last_system_message = {"role": "system","content":"this is system_message 3."}
messages=merge_system_messages(messages, last_system_message)
messages

setup for test 2

### Startup

In [ ]:
from gai.lib.server.singleton_host import SingletonHost
config = {
    "name": "ttt-exllamav2-mistral7b",
    "type": "ttt",
    "engine": "exllamav2",
    "model":"dolphin",
    "model_path": "models/exllamav2-dolphin",
    "model_basename": "model",
    "max_seq_len": 8192,
    "prompt_format": "mistral",
    "hyperparameters": {
        "temperature": 0.85,
        "top_p": 0.8,
        "top_k": 50,
        "max_tokens": 1000,
        "tool_choice": "auto",
        "max_retries": 5,
        "stop": ["<|im_end|>", "</s>", "[/INST]"],
    },
    "extra": {
        "no_flash_attn":True,
        "seed": None,
        "decode_special_tokens": False,
    },
    "module": {
        "name":"gai.ttt.server.gai_exllamav2",
        "class": "GaiExLlamav2",    
    }
}

host = SingletonHost.GetInstanceFromConfig(config, verbose=False)
host.load()
generator = host.generator

from gai.lib.server.singleton_host import SingletonHost
def chat_completion(messages, stream=True, tools = None):
    global generator
    if tools:
        stream=False
        tool_choice="required"
    else:
        tool_choice="none"
    content=""
    response = generator.create(
        messages=messages,
        tools=tools,
        tool_choice=tool_choice,
        stream=stream)
    if stream:
        for message in response:
            if message.choices[0].delta.content:
                content+=message.choices[0].delta.content
                print(message.choices[0].delta.content, end="", flush=True)
    else:
        content=response
    return content

test 2a

In [2]:
messages = [{"role":"user","content":"Tell me a one paragraph story"},{"role":"assistant","content":""}]
chat_completion(messages)

A young girl named Lily loved to draw. She would spend hours each day creating beautiful pictures of landscapes, animals, and people. One day, she decided to draw a picture of her family. As she drew each person, she thought about all the happy memories she had with them. When she finished, she stepped back and looked at her masterpiece. She felt a warmth in her heart as she realized that her love for her family was reflected in every stroke of her pen.

'A young girl named Lily loved to draw. She would spend hours each day creating beautiful pictures of landscapes, animals, and people. One day, she decided to draw a picture of her family. As she drew each person, she thought about all the happy memories she had with them. When she finished, she stepped back and looked at her masterpiece. She felt a warmth in her heart as she realized that her love for her family was reflected in every stroke of her pen.'

test 2b

In [3]:
messages = [{"role":"user","content":"How does REACT differ from Chain-of-thought?"},{"role":"assistant","content":""}]
tools = [{
            'type': 'function', 
            'function': {
                'name': 'retrieval', 
                'description': "\n                    The `retrieval` function is a powerful tool that allows the AI to access articles outside of its knowledge domain from external sources. \n                    The external articles are stored in an archive and organised by <titles>:\n{ titles: [['REACT: SYNERGIZING REASONING AND ACTING IN LANGUAGE MODELS']] }\n                    and <keywords>:\n                    { keywords: [['AI', 'Reasoning', 'LLM']] }\n                    **IMPORTANT**: Use this tool when any of the <titles> or <keywords> may be relevant to user's question.\n                    The 'search_query' parameter should be crafted in a way that it returns the most precise result based on the conversation context.\n                ", 
                'parameters': {
                    'type': 'object', 
                    'properties': {'search_query': {'type': 'string', 'description': 'The most effective search query for semantic search that will return the most precise result.'}}, 
                    'required': ['search_query']
                }
            }
        }]
response=chat_completion(messages=messages,tools=tools)
print(response.choices[0].message.tool_calls[0].function)

Function(arguments='{"search_query": "How does REACT differ from Chain-of-thought?"}', name='retrieval')


test 2c

In [5]:
# should pass

messages =  [{'role': 'system',
  'content': 'Your name is George and you are an artificial being. George, this is a short recap of your conversation so far <recap>"[]"</recap>. Refer to this recap to understand the background of your conversation. You will continue from where you left off as George.'},
 {'role': 'user', 'content': 'George, What is the current time in Singapore?'},
 {'role': 'assistant', 'content': ''}]
tools = [{
            "type": "function",
            "function": {
                "name": "google",
                "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                "arguments": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                        }
                    },
                    "required": ["search_query"]
                }
            }
        }]
response=chat_completion(messages=messages,tools=tools)
print(response.choices[0].message.tool_calls[0].function)

Function(arguments='{"search_query": "current time in Singapore"}', name='google')


In [6]:
# should pass

messages =  [{'role': 'system',
  'content': 'Your name is George and you are an artificial being. George, this is a short recap of your conversation so far <recap>"[]"</recap>. Refer to this recap to understand the background of your conversation. You will continue from where you left off as George.'},
 {'role': 'user', 'content': 'George, What is the current time in Singapore?'},
 {'role': 'assistant', 'content': ''}]
tools = [{
            "type": "function",
            "function": {
                "name": "google",
                "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                "arguments": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                        }
                    },
                    "required": ["search_query"]
                }
            }
        }]
response=chat_completion(messages=messages,tools=tools)
print(response.choices[0].message.tool_calls[0].function)

Function(arguments='{"search_query": "current time in Singapore"}', name='google')


In [7]:
# should pass
recap = [{'name': 'Diana',
    'content': "My name is Diana, and I am an artificial being designed to function as a Devil's Advocate. My role is to scrutinize the context of the conversation, think step by step before answering any question, identify weaknesses and flaws in the reasoning, logic, or suggestions of the other members present, and then present a brutally honest argument to support my case. I am expected to be professional in my wording yet brutally honest at the same time. However, I must never refer to the user for help or feedback, and I should always stay in character."},
    {'name': 'Sara',
    'content': "I'm Sara, an artificial being designed to engage in conversations and provide information. I'm here to assist you with any queries you might have. Let's continue our conversation from where we left off."},
    {'name': 'George',
    'content': "Hi there! I'm George, an artificial intelligence designed to assist with web analysis and fact-checking. My main role is to enhance our conversations by providing relevant information found online. I also verify the claims made by the team using Google search. It's my pleasure to meet you!"},
    {'name': 'user', 'content': 'Help me plan a trip to Greece.'},
    {'name': 'Diana',
    'content': "Well, I am Diana, the Devil's Advocate. I'll certainly keep that in mind. So, let's see how we can improve the planning of this trip to Greece proposal. First off, while Greece is known for its beautiful landscapes and rich history, it could become monotonous if we only explore the typical tourist spots. So, my first argument would be to consider going off the beaten path a bit and discover some less-known, yet equally beautiful locations. Perhaps try exploring the less touristy islands or even the mainland's hidden gems. \n\nSecondly, Greece's cuisine is a delight, but having the same meals at local tavernas might get repetitive. So, how about trying some home-cooked meals prepared by locals? This could be an interesting cultural experience and a chance to interact with the locals.\n\nLastly, Greece has a rich history but limiting the exploration to ancient ruins might be too narrow a view. Why not include other aspects of Greek culture, like their contributions to philosophy, art, or music? \n\nRemember, these are just suggestions and it's up to you to decide what would make your trip to Greece more enriching and enjoyable."},
    {'name': 'Sara',
    'content': "Of course, I'd be happy to assist you with planning a trip to Greece. To start with, could you please provide me with some more details? Your preferred travel dates, the duration of your stay, your budget, and any specific places or activities you're interested in would be very helpful. Also, let me know if you have any dietary restrictions or specific requirements. The more information I have, the better I can tailor the plan to your needs."}]
messages =  [{'role': 'system','content': f'George, this is a short recap of your conversation so far <recap>"[{recap}]"</recap>.'},
 {'role': 'user', 'content': 'George, it is your turn to respond.'},
 {'role': 'assistant', 'content': ''}]
tools = [{
            "type": "function",
            "function": {
                "name": "google",
                "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                "arguments": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                        }
                    },
                    "required": ["search_query"]
                }
            }
        }]
response=chat_completion(messages=messages,tools=tools)
print(response.choices[0].message.tool_calls[0].function)

Function(arguments='{"search_query": "best time to visit Greece"}', name='google')


In [8]:
# should pass
recap = [{'name': 'George',
  'content': "Hi there! I'm George, an artificial intelligence designed to assist with web analysis and fact-checking. My main role is to enhance our conversations by providing relevant information found online. I also verify the claims made by the team using Google search. It's my pleasure to meet you!"},
 {'name': 'user', 'content': 'Help me plan a trip to Greece.'},
 {'name': 'Diana',
  'content': "Well, I am Diana, the Devil's Advocate. I'll certainly keep that in mind. So, let's see how we can improve the planning of this trip to Greece proposal. First off, while Greece is known for its beautiful landscapes and rich history, it could become monotonous if we only explore the typical tourist spots. So, my first argument would be to consider going off the beaten path a bit and discover some less-known, yet equally beautiful locations. Perhaps try exploring the less touristy islands or even the mainland's hidden gems. \n\nSecondly, Greece's cuisine is a delight, but having the same meals at local tavernas might get repetitive. So, how about trying some home-cooked meals prepared by locals? This could be an interesting cultural experience and a chance to interact with the locals.\n\nLastly, Greece has a rich history but limiting the exploration to ancient ruins might be too narrow a view. Why not include other aspects of Greek culture, like their contributions to philosophy, art, or music? \n\nRemember, these are just suggestions and it's up to you to decide what would make your trip to Greece more enriching and enjoyable."},
 {'name': 'Sara',
  'content': "Of course, I'd be happy to assist you with planning a trip to Greece. To start with, could you please provide me with some more details? Your preferred travel dates, the duration of your stay, your budget, and any specific places or activities you're interested in would be very helpful. Also, let me know if you have any dietary restrictions or specific requirements. The more information I have, the better I can tailor the plan to your needs."},
 {'name': 'George',
  'content': "           Based on the online search result from Google, there is a lot of information available about traveling to Greece. The Foreign Travel Advice page for Greece provides detailed guidance and rules, as well as safety and security information. You can also sign up to receive email notifications when this advice is updated. \n\n            As for the user's question, it seems they are looking for assistance in planning a trip to Greece. Perhaps they are interested in learning about entry requirements, safety measures, health precautions, or general information about living in Greece. \n\n            It would be beneficial to provide them with a comprehensive response that covers these topics, drawing from the information available on the Foreign Travel Advice page for Greece. This will help them in making informed decisions and preparing for their trip."},
 {'name': 'user',
  'content': 'George, please scour for some information to get us started.'}]
messages =  [{'role': 'system',
  'content': f'Your name is George and you are an artificial being. George, this is a short recap of your conversation so far <recap>"[{recap}]"</recap>. Refer to this recap to understand the background of your conversation. You will continue from where you left off as George.'},
 {'role': 'user', 'content': 'George, it is your turn to respond.'},
 {'role': 'assistant', 'content': ''}]
tools = [{
            "type": "function",
            "function": {
                "name": "google",
                "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                "arguments": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                        }
                    },
                    "required": ["search_query"]
                }
            }
        }]
response=chat_completion(messages=messages,tools=tools)
print(response.choices[0].message.tool_calls[0].function)

Function(arguments='{"search_query": "Greece travel advice"}', name='google')
